In [5]:
import numpy as np
import sys
import glob 
import os 
from pathlib import Path

sys.path.insert(0, 'C:\\Suite2P\\')
import suite2p

from suite2p.run_s2p import run_s2p

mouseName = 'GF208'

fast_diskpath = str(Path('D:\Foustoukos'))
save_path = str(Path('D:\Foustoukos' + '\\' +  mouseName))

print(type(fast_diskpath))

ops = {
        # file paths
        'look_one_level_down': False, # whether to look in all subfolders when searching for tiffs
        'fast_disk': fast_diskpath, # used to store temporary binary file, defaults to save_path0
        'delete_bin': False, # whether to delete binary file after processing
        'mesoscan': False, # for reading in scanimage mesoscope files
        'h5py': [], # take h5py as input (deactivates data_path)
        'h5py_key': 'data', #key in h5py where data array is stored
        'save_path0': save_path, # stores results, defaults to first item in data_path
        'subfolders': [],
        'diameter' : 15,
        # main settings
        'nplanes' : 1, # each tiff has these many planes in sequence
        'nchannels' : 1, # each tiff has these many channels per plane
        'functional_chan' : 1, # this channel is used to extract functional ROIs (1-based)
        'tau':  0.7, # this is the main parameter for deconvolution
        'fs': 30.,  # sampling rate (PER PLANE - e.g. if you have 12 planes then this should be around 2.5)
        'force_sktiff': False, # whether or not to use scikit-image for tiff reading
        # output settings
        'preclassify': 0.5, # apply classifier before signal extraction with probability 0.5 (turn off with value 0)
        'save_mat': True, # whether to save output as matlab files
        'combined': True, # combine multiple planes into a single result /single canvas for GUI
        'aspect': 1.0, # um/pixels in X / um/pixels in Y (for correct aspect ratio in GUI)
        # bidirectional phase offset
        'do_bidiphase': False,
        'bidiphase': 0,
        # registration settings
        'do_registration': 1, # whether to register data (2 forces re-registration)
        'keep_movie_raw': False,
        'nimg_init': 300, # subsampled frames for finding reference image
        'batch_size': 500, # number of frames per batch
        'maxregshift': 0.1, # max allowed registration shift, as a fraction of frame max(width and height)
        'align_by_chan' : 1, # when multi-channel, you can align by non-functional channel (1-based)
        'reg_tif': False, # whether to save registered tiffs
        'reg_tif_chan2': False, # whether to save channel 2 registered tiffs
        'subpixel' : 10, # precision of subpixel registration (1/subpixel steps)
        'smooth_sigma': 1.15, # ~1 good for 2P recordings, recommend >5 for 1P recordings
        'th_badframes': 1.0, # this parameter determines which frames to exclude when determining cropping - set it smaller to exclude more frames
        'pad_fft': False,
        # non rigid registration settings
        'nonrigid': True, # whether to use nonrigid registration
        'block_size': [128, 128], # block size to register (** keep this a multiple of 2 **)
        'snr_thresh': 1.2, # if any nonrigid block is below this threshold, it gets smoothed until above this threshold. 1.0 results in no smoothing
        'maxregshiftNR': 5, # maximum pixel shift allowed for nonrigid, relative to rigid
        # 1P settings
        '1Preg': False, # whether to perform high-pass filtering and tapering
        'spatial_hp': 50, # window for spatial high-pass filtering before registration
        'pre_smooth': 2, # whether to smooth before high-pass filtering before registration
        'spatial_taper': 50, # how much to ignore on edges (important for vignetted windows, for FFT padding do not set BELOW 3*ops['smooth_sigma'])
        # cell detection settings
        'roidetect': True, # whether or not to run ROI extraction
        'spatial_scale': 0, # 0: multi-scale; 1: 6 pixels, 2: 12 pixels, 3: 24 pixels, 4: 48 pixels
        'connected': True, # whether or not to keep ROIs fully connected (set to 0 for dendrites)
        'nbinned': 5000, # max number of binned frames for cell detection
        'max_iterations': 20, # maximum number of iterations to do cell detection
        'threshold_scaling': 5., # adjust the automatically determined threshold by this scalar multiplier
        'max_overlap': 0.75, # cells with more overlap than this get removed during triage, before refinement
        'high_pass': 100, # running mean subtraction with window of size 'high_pass' (use low values for 1P)
        # ROI extraction parameters
        'inner_neuropil_radius': 2, # number of pixels to keep between ROI and neuropil donut
        'min_neuropil_pixels': 350, # minimum number of pixels in the neuropil
        'allow_overlap': False, # pixels that are overlapping are thrown out (False) or added to both ROIs (True)
        # channel 2 detection settings (stat[n]['chan2'], stat[n]['not_chan2'])
        'chan2_thres': 0.65, # minimum for detection of brightness on channel 2
        # deconvolution settings
        'baseline': 'maximin', # baselining mode (can also choose 'prctile')
        'win_baseline': 60., # window for maximin
        'sig_baseline': 10., # smoothing constant for gaussian filter
        'prctile_baseline': 8.,# optional (whether to use a percentile baseline)
        'neucoeff': .7,  # neuropil coefficient
        'xrange': np.array([0, 0]),
        'yrange': np.array([0, 0]),
      }




<class 'str'>


In [6]:
# provide an h5 path in 'h5py' or a tiff path in 'data_path'
# db overwrites any ops (allows for experiment specific settings)

datapath = Path('\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\' + mouseName + '\\Recordings\\Imaging\\')
datapath = str(datapath)
imaging_days = glob.glob(os.path.join(datapath, "GF*"))
binpath = Path('D:\Foustoukos\BIN')
binpath = str(binpath)

for n,f in enumerate(imaging_days):
    imaging_days[n] = os.path.basename(f)
    
    
for n in range(len(imaging_days)):
    
    current_day_path = str(datapath) + '\\' + str(imaging_days[n]) +'\\'
    
    binpath = Path('D:\Foustoukos\BIN' + '\\' + mouseName + '\\' + str(imaging_days[n]))
    
    binpath = str(binpath)
    
    savepath = Path('D:\Foustoukos' + '\\' +  mouseName + '\\' + str(imaging_days[n]))
    
    savepath = str(savepath)
   
    print(current_day_path)
    
    db = {
          'h5py': [], # a single h5 file path
          'h5py_key': 'data',
          'look_one_level_down': False, # whether to look in ALL subfolders when searching for tiffs
          'data_path': [current_day_path], # a list of folders with tiffs 
                                                 # (or folder of folders with tiffs if look_one_level_down is True, or subfolders is not empty)

          'subfolders': [], # choose subfolders of 'data_path' to look in (optional)
          'fast_disk': binpath, # string which specifies where the binary file will be stored (should be an SSD)
          'save_path0' : savepath,
        }


    #print(db['data_path'])
    
    opsEnd=run_s2p(ops=ops,db=db)




\\sv1files.epfl.ch\Petersen-Lab\data\GF208\Recordings\Imaging\GF208_03102019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF208\\Recordings\\Imaging\\GF208_03102019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF208\\GF208_03102019', 'save_path0': 'D:\\Foustoukos\\GF208\\GF208_03102019'}
FOUND OPS IN D:\Foustoukos\GF208\GF208_03102019\suite2p\plane0
FOUND BINARIES: D:\Foustoukos\BIN\GF208\GF208_03102019\suite2p\plane0\data.bin
SKIPPING REGISTRATION FOR ALL PLANES...
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 43
Binned movie [5037,503,499], 235.87 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 4.20, threshold 41.97 
0 ROIs, score=652.06
Found 772 ROIs, 385.58 sec
NOTE: applying classifier C:\Suite2P\suite2p\classifiers/classifier_user.npy
After classification with threshold 0.50, 336 ROIs remain
After re

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 5.04 sec.
Extracted fluorescence from 333 ROIs in 216606 frames, 502.80 sec.
----------- Total 905.21 sec.
----------- SPIKE DECONVOLUTION
----------- Total 114.90 sec.
Plane 0 processed in 1026.33 sec (can open in GUI).
total = 1027.20 sec.
TOTAL RUNTIME 1028.54 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF208\Recordings\Imaging\GF208_04102019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF208\\Recordings\\Imaging\\GF208_04102019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF208\\GF208_04102019', 'save_path0': 'D:\\Foustoukos\\GF208\\GF208_04102019'}
FOUND OPS IN D:\Foustoukos\GF208\GF208_04102019\suite2p\plane0
FOUND BINARIES: D:\Foustoukos\BIN\GF208\GF208_04102019\suite2p\plane0\data.bin
SKIPPING REGISTRATION FOR ALL PLANES...
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- ROI DETECTION AND EXTRACTION
Binning movie in chunks of length 31
Binned movie [5078,500,499]

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 5.04 sec.
Extracted fluorescence from 354 ROIs in 157426 frames, 369.97 sec.
----------- Total 714.73 sec.
----------- SPIKE DECONVOLUTION
----------- Total 91.85 sec.
Plane 0 processed in 811.39 sec (can open in GUI).
total = 812.16 sec.
TOTAL RUNTIME 813.18 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF208\Recordings\Imaging\GF208_29092019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF208\\Recordings\\Imaging\\GF208_29092019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF208\\GF208_29092019', 'save_path0': 'D:\\Foustoukos\\GF208\\GF208_29092019'}
** Found 516 tifs - converting to binary **
time 2715.94 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 8.48 sec.
2500/147041 frames, 74.26 sec.
5000/147041 frames, 141.34 sec.
7500/147041 frames, 206.17 sec.
10000/147041 frames, 274.96 sec.
12500/147041 

...

Masks made in 2.53 sec.
Extracted fluorescence from 168 ROIs in 102327 frames, 198.31 sec.
----------- Total 413.13 sec.
----------- SPIKE DECONVOLUTION
----------- Total 39.52 sec.
Plane 0 processed in 3146.30 sec (can open in GUI).
total = 4817.30 sec.
TOTAL RUNTIME 4817.92 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF208\Recordings\Imaging\GF208_01102019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF208\\Recordings\\Imaging\\GF208_01102019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF208\\GF208_01102019', 'save_path0': 'D:\\Foustoukos\\GF208\\GF208_01102019'}
** Found 741 tifs - converting to binary **
154000 frames of binary, time 2474.00 sec.
time 3339.51 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 8.36 sec.
2500/204981 frames, 68.14 sec.
5000/204981 frames, 133.01 sec.
7500/204981 frames, 198.96 sec.


C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 4.50 sec.
Extracted fluorescence from 303 ROIs in 204981 frames, 454.91 sec.
----------- Total 832.14 sec.
----------- SPIKE DECONVOLUTION
----------- Total 101.10 sec.
Plane 0 processed in 6139.25 sec (can open in GUI).
total = 9478.76 sec.
TOTAL RUNTIME 9480.19 sec
\\sv1files.epfl.ch\Petersen-Lab\data\GF208\Recordings\Imaging\GF208_02102019\
{'h5py': [], 'h5py_key': 'data', 'look_one_level_down': False, 'data_path': ['\\\\sv1files.epfl.ch\\Petersen-Lab\\data\\GF208\\Recordings\\Imaging\\GF208_02102019\\'], 'subfolders': [], 'fast_disk': 'D:\\Foustoukos\\BIN\\GF208\\GF208_02102019', 'save_path0': 'D:\\Foustoukos\\GF208\\GF208_02102019'}
** Found 928 tifs - converting to binary **
8000 frames of binary, time 105.05 sec.
224000 frames of binary, time 3679.20 sec.
time 4176.50 sec. Wrote tifs to binaries for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
----------- REGISTRATION
Reference frame, 9.54 sec.
2500/254556 frames, 74.15 sec.
5000/254556 frames, 148

C:\Users\foustouk\.conda\envs\suite2p\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Masks made in 4.53 sec.
Extracted fluorescence from 318 ROIs in 254556 frames, 627.27 sec.
----------- Total 1082.36 sec.
----------- SPIKE DECONVOLUTION
----------- Total 135.02 sec.
Plane 0 processed in 8424.99 sec (can open in GUI).
total = 12601.49 sec.
TOTAL RUNTIME 12603.06 sec


In [4]:
from natsort import natsorted, ns
import glob 
import os 
mouseName = 'GF207'
datapath = '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\' + mouseName + '\\'
datapath = str(datapath)
imaging_days =natsorted(glob.glob(os.path.join(datapath, "GF*")))


In [10]:
imaging_days

['\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00001.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00002.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00003.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00004.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00005.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00006.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF207_187um_920nm_1x_01102019_00007.tif',
 '\\\\sv1files.epfl.ch\\Petersen-Lab\\analysis\\Georgios_Foustoukos\\RawDataAll\\GF207\\GF